In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import random
import time

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [30]:
data = pd.read_csv('https://raw.githubusercontent.com/aidev1065/eeg-eye-state-classification/master/eeg_eye_state.csv', skiprows = [i for i in range(19)], header=None)
data.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
14957,4281.03,3990.26,4245.64,4116.92,4333.85,4614.36,4074.87,4625.64,4203.08,4221.54,4171.28,4269.23,4593.33,4340.51,1
14958,4276.92,3991.79,4245.13,4110.77,4332.82,4615.38,4073.33,4621.54,4194.36,4217.44,4162.56,4259.49,4590.26,4333.33,1
14959,4277.44,3990.77,4246.67,4113.85,4333.33,4615.38,4072.82,4623.59,4193.33,4212.82,4160.51,4257.95,4591.79,4339.49,1
14960,4284.62,3991.79,4251.28,4122.05,4334.36,4616.41,4080.51,4628.72,4200.00,4220.00,4165.64,4267.18,4596.41,4350.77,1
14961,4287.69,3997.44,4260.00,4121.03,4333.33,4616.41,4088.72,4638.46,4212.31,4226.67,4167.69,4274.36,4597.95,4350.77,1


In [31]:
X = df[:,:14]
y = df[:,14:]

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1234)

In [33]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [34]:
print(f"Training Data has {len(X_train)} examples")
print(f"Testing Data has {len(X_test)} examples")

Training Data has 11221 examples
Testing Data has 3741 examples


In [35]:
print(X_train)

[[ 1.66689168  1.99722465  2.47569811 ...  2.30736168  1.47897885
   0.00486389]
 [-0.15840629 -0.45475565 -0.24340696 ... -0.01770198 -0.34797345
  -0.01269127]
 [-0.30441115 -0.16683766 -0.05414157 ... -0.1983419  -0.1243303
  -0.01299245]
 ...
 [ 1.1801561  -0.52938566  0.21765367 ...  0.89794746  1.83683698
   0.00674737]
 [-0.26375758 -0.11361961 -0.40892772 ... -0.41515679 -0.27337726
  -0.01186266]
 [ 2.01568984  2.42359274  1.15176251 ...  0.4762977   0.44248442
   0.0036577 ]]


In [36]:
## Train Data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
## Test Data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = testData(torch.FloatTensor(X_test))

In [91]:
# Create DataLoaders for torch
BATCH_SIZE = 64
LEARNING_RATE=0.001
EPOCHS=50

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [122]:
class MLPNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer_1 = nn.Linear(14, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1)
        
        self.leakyRelu = nn.LeakyReLU()
        self.dropout = nn.Dropout(p=0.25)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)

    def forward(self, x):
        x = self.leakyRelu(self.layer_1(x))
        x = self.batchnorm1(x)
        x = self.dropout(x)
        x = self.leakyRelu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        return x

model = MLPNetwork()

In [123]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,441 trainable parameters


In [124]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [125]:
model = model.to(device)
criterion = criterion.to(device)

In [126]:
def accuracy(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [127]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        acc = accuracy(y_pred, y_batch)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.63393 | Acc: 63.699
Epoch 002: | Loss: 0.57673 | Acc: 69.392
Epoch 003: | Loss: 0.54671 | Acc: 71.790
Epoch 004: | Loss: 0.52014 | Acc: 74.091
Epoch 005: | Loss: 0.50304 | Acc: 75.097
Epoch 006: | Loss: 0.47587 | Acc: 76.705
Epoch 007: | Loss: 0.45316 | Acc: 78.426
Epoch 008: | Loss: 0.44924 | Acc: 78.494
Epoch 009: | Loss: 0.44486 | Acc: 78.756
Epoch 010: | Loss: 0.43909 | Acc: 78.847
Epoch 011: | Loss: 0.41929 | Acc: 80.307
Epoch 012: | Loss: 0.41406 | Acc: 80.307
Epoch 013: | Loss: 0.41286 | Acc: 80.324
Epoch 014: | Loss: 0.41181 | Acc: 80.710
Epoch 015: | Loss: 0.40170 | Acc: 80.977
Epoch 016: | Loss: 0.39976 | Acc: 81.301
Epoch 017: | Loss: 0.39273 | Acc: 81.784
Epoch 018: | Loss: 0.39467 | Acc: 81.500
Epoch 019: | Loss: 0.39490 | Acc: 81.750
Epoch 020: | Loss: 0.38203 | Acc: 82.585
Epoch 021: | Loss: 0.38697 | Acc: 82.148
Epoch 022: | Loss: 0.37754 | Acc: 82.472
Epoch 023: | Loss: 0.37203 | Acc: 82.795
Epoch 024: | Loss: 0.37351 | Acc: 82.909
Epoch 025: | Los

In [128]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [129]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

         0.0       0.66      0.42      0.52      2083
         1.0       0.50      0.73      0.59      1658

    accuracy                           0.56      3741
   macro avg       0.58      0.58      0.56      3741
weighted avg       0.59      0.56      0.55      3741

